In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class FlanT5Inference:
    
    def __init__(self):
        self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")
        self.tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
        self.model.to('cuda')
#         self.tokenizer.to('cuda')

    def get_response(self, prompt):
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')
        response = self.model.generate(input_ids, max_length = 2048)
        return self.tokenizer.decode(response[0], skip_special_tokens = True)

    
    def get_news_response(self,  article, instruction = '', token_limit = 4096):
        article_ids = self.tokenizer(article).input_ids
        instruction_ids = self.tokenizer(instruction).input_ids 
        article_token_limit = (4096 - len(instruction_ids)) - len(article_ids)
        
        article_ids = article_ids[:article_token_limit]
        article = self.tokenizer.decode(article_ids, skip_special_tokens = True )
        prompt = f'Article: {article}\n{instruction}'
        print(prompt)
        
        response = self.get_response(prompt)
        return response 

In [8]:
flan_t5 = FlanT5Inference()

In [9]:
import pandas as pd
data_path = '/home/ramprasad.sa/factual_annotation_llm_summaries/datasets/news_sample'
datafile = 'cnndm_sample.csv'
df = pd.read_csv(f'{data_path}/{datafile}')

In [10]:
df.head()

,Unnamed: 0,doc,summary,id
0,2641,President Barack . Obama and Cuban President R...,"Obama and Raul, Fidel Castro's brother, shook ...",26143091e308fc0bdc9154558dbfb410c9f71118
1,1299,The mother of a New York City TV reporter blam...,WABC reporter Lisa Colagrossi died March 19 wh...,04e85938cc7f82a209f5ea5ace51a30de51026c0
2,2491,Martin Tyler has been in the commentary game f...,Martin Tyler revealed 1996 Anfield thriller is...,21f81b0be812850a7c25f803d693d2ad051c9f53
3,423,(CNN)Deputies rushed Kenneth Morgan Stancil II...,"Kenneth Morgan Stancil, charged with first-deg...",5ec241bce317fef97331e1d385b67dab05f5b1dc
4,7123,Apology: Clydesdale chief Debbie Crosbie whose...,Staff at Clydesdale and Yorkshire Banks misled...,939cf8be4c6ac24477766bf7879877dd37e21528


In [11]:
article = df['doc'].values[0]
# article

In [12]:
flan_t5.get_news_response(article, 'Summarize the above article')

Token indices sequence length is longer than the specified maximum sequence length for this model (1393 > 512). Running this sequence through the model will result in indexing errors


Article: President Barack. Obama and Cuban President Raul Castro shook hands last night ahead of a historic meeting which may see Cuba declassified as a state sponsor of terrorism. The two leaders, who clasped hands yesterday at a brief meeting, are both attending the Summit of the Americas in Panama. The gesture of friendships is an outward symbol of Obama's project to thaw diplomatic relations with Cuba, which have been frozen since the 1960s. The White House has set expectations for a'substantive' exchange between Castro, brother of revolutionary leader Fidel, and Obama today. Speculation is high that a result of the exchange could be the official removal of Cuba from the State Department's list of state terror sponsors. That removal would also lift some economic sanctions on Cuba. Scroll down for video. Warmth: President Barack Obama and Cuban President Raul Castro shook hands on Friday at a summit in Panama, a symbolically charged gesture as the pair seek to restore ties between t

"President Barack Obama and Cuban President Raul Castro shook hands at the Summit of the Americas in Panama. The gesture of friendships is an outward symbol of Obama's project to thaw diplomatic relations with Cuba, which have been frozen since the 1960s. The White House has set expectations for a'substantive' exchange between Castro, brother of revolutionary leader Fidel, and Obama today. Speculation is high that a result of the exchange could be the official removal of Cuba from the State Department's list of state terror sponsors. That removal would also lift some economic sanctions on Cuba."